In [1]:
import json
import requests
import random
import string 
import secrets 
from tqdm import tqdm 
import time
import re
import os
import collections
import torch
import torch.nn as nn
import numpy as np
import math

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [12]:
BASE_MODEL_PATH = "/kaggle/input/hman-model-62-75/bilstm_attn_hangman_GATED_MLPGATE_finetuned_62.75.pth"
FINETUNED_SHORT_MODEL_PATH = "/kaggle/input/hman-shortlen-model/bilstm_attn_hangman_GATED_MLPGATE_short_words_len5.pth"

SHORT_WORD_LENGTH_THRESHOLD = 5
# alpha for DAWG fusion , used for word_len > 5
BASE_ALPHA_VALUE = 0.365         

# configs
SEED = 42
D_MODEL = 128
HIDDEN_DIM = 256
NUM_LAYERS = 4
ATTN_HEADS = 8
MAX_SEQ_LEN = 32 
ALPHABET_LEN = 26
STATE_DIM_HISTORY = ALPHABET_LEN * 2

CHAR_TO_IX = {char: i + 2 for i, char in enumerate("abcdefghijklmnopqrstuvwxyz")}
CHAR_TO_IX['_'] = 1
VOCAB_SIZE = len(CHAR_TO_IX) + 1 # 28
IX_TO_CHAR = {i: char for char, i in CHAR_TO_IX.items()}
IX_TO_CHAR[0] = '<pad>'
ALL_LETTERS = set("abcdefghijklmnopqrstuvwxyz")

# model arch
class Gated_BiLSTM_Attention_Hangman(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, state_dim, num_heads):
        super(Gated_BiLSTM_Attention_Hangman, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm_output_dim = hidden_dim * 2
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.attention = nn.MultiheadAttention(embed_dim=self.lstm_output_dim, num_heads=num_heads, batch_first=True)
        self.norm = nn.LayerNorm(self.lstm_output_dim)
        self.state_projection = nn.Sequential(
            nn.Linear(state_dim, self.lstm_output_dim), nn.ReLU(),
            nn.Linear(self.lstm_output_dim, self.lstm_output_dim))
        self.gate_fc = nn.Sequential(
            nn.Linear(self.lstm_output_dim * 2, self.lstm_output_dim), nn.ReLU(),
            nn.Linear(self.lstm_output_dim, self.lstm_output_dim), nn.Sigmoid())
        self.fc = nn.Linear(self.lstm_output_dim, vocab_size)

    def forward(self, x, state):
        attn_key_padding_mask, embedded, seq_len = (x == 0), self.embedding(x), x.size(1)
        lstm_out, _ = self.lstm(embedded)
        # this handles potential empty sequences if input is just padding after filtering
        if lstm_out.size(1) == 0:
             print(f"Warning: LSTM output has sequence length 0 for input shape {x.shape}")
             # for returning zero logits for the expected output shape
             batch_size = x.size(0)
             # Ensure at least 1 for output dim
             output_seq_len = max(1, seq_len) 
             return torch.zeros(batch_size, output_seq_len, VOCAB_SIZE, device=x.device)

        attn_output, _ = self.attention(lstm_out, lstm_out, lstm_out, key_padding_mask=attn_key_padding_mask[:, :lstm_out.size(1)]) # Adjust mask length if needed
        O_Attn = self.norm(lstm_out + attn_output)
        C_State_projected = self.state_projection(state)
        
        # ensures C_State matches LSTM output seq len
        C_State = C_State_projected.unsqueeze(1).repeat(1, O_Attn.size(1), 1)
        combined_features = torch.cat([O_Attn, C_State], dim=-1)
        Gamma = self.gate_fc(combined_features)
        O_Gated = Gamma * O_Attn + (1 - Gamma) * C_State
        return self.fc(O_Gated)


# hybrid stategy
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        try:
             full_dictionary_location = "/kaggle/input/words-250000-train/words_250000_train.txt"
             
             self.full_dictionary = self.build_dictionary(full_dictionary_location)
             print(f"Loaded dictionary with {len(self.full_dictionary)} words.")
        except FileNotFoundError:
             print("ERROR: Could not find dictionary file. Ensure 'words_250000_train.txt' is accessible.")
             raise

        self.base_alpha = BASE_ALPHA_VALUE
        print(f"Initialized HYBRID strategy. Base Alpha = {self.base_alpha}")

        # Seed
        print("Setting up device and seeding")
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        random.seed(SEED)
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(SEED)

        # base model
        print("Initializing BASE Hangman Model.")
        self.base_model = self._load_single_model(BASE_MODEL_PATH)
        print("BASE model loaded successfully.")

        # base model fine-tuned on short words ( len<5 ) , although multiple models could be used depending 
        # on the length bin , to make the models length specific.  
        print("Initializing FINE-TUNED SHORT Hangman Model...")
        self.short_model = self._load_single_model(FINETUNED_SHORT_MODEL_PATH)
        print("SHORT model loaded successfully.")

    def _load_single_model(self, model_path):
        """ Helper to load a single model instance. """
        model = Gated_BiLSTM_Attention_Hangman(
            VOCAB_SIZE, D_MODEL, HIDDEN_DIM, NUM_LAYERS,
            state_dim=STATE_DIM_HISTORY, num_heads=ATTN_HEADS
        ).to(self.device)
        if not os.path.exists(model_path):
              raise FileNotFoundError(f"Required model file '{model_path}' not found.")

        print(f" -> Loading weights from {model_path}.")
        try:
            # Loads the state dict
            state_dict = torch.load(model_path, map_location=self.device)
            
            model.load_state_dict(state_dict)
            model.eval()
            return model
        except Exception as e:
            print(f"ERROR loading model state_dict from {model_path}: {e}")
            raise

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):
        """ Guesses a letter using the HYBRID strategy. """
        pattern = word.replace(" ", "")
        guessed_letters_set = set(self.guessed_letters)
        word_length = len(pattern)
        unguessed = ALL_LETTERS - guessed_letters_set

        # hybrid logic
        if word_length <= SHORT_WORD_LENGTH_THRESHOLD:
            # Uses fine-tuned short words model
            lstm_probs = self._get_lstm_probs(pattern, guessed_letters_set, model_to_use=self.short_model)
            final_scores = lstm_probs # No fusion for this case
            current_alpha = 0.0
        else:
            # Uses Base Model + Adaptive DAWG Fusion
            lstm_probs = self._get_lstm_probs(pattern, guessed_letters_set, model_to_use=self.base_model)
            incorrect_guesses = guessed_letters_set - {c for c in pattern if c.isalpha()}
            valid_words = self._filter_words_py(pattern, incorrect_guesses)
            dawg_probs = self._get_dawg_probs(valid_words, guessed_letters_set)

            # Adaptive Alpha Logic
            num_blanks = pattern.count('_')
            if num_blanks == 1:
                current_alpha = 0.0 # Trusting LSTM more 
            else:
                current_alpha = self.base_alpha

            # Fuse the scores
            final_scores = {}
            for letter in unguessed:
                lstm_p = lstm_probs.get(letter, 0.0)
                dawg_p = dawg_probs.get(letter, 0.0)
                final_scores[letter] = (1.0 - current_alpha) * lstm_p + current_alpha * dawg_p

        # Choose the best letter 
        best_letter, max_score = None, -1.0
        # Iterates through unguessed letters only
        possible_guesses = list(unguessed)
        
        # Shuffles to break ties randomly
        random.shuffle(possible_guesses) 

        for letter in possible_guesses:
            score = final_scores.get(letter, 0.0)
            if score > max_score:
                max_score = score
                best_letter = letter

        # fallback
        if best_letter is not None:
            guess_letter = best_letter
        else:
            # based on general letter frequency
            fallback_order = "esiarntolcdugpmhbyfvwkxqz"
            for letter in fallback_order:
                if letter in unguessed: 
                    # Checks if the fallback letter is actually available
                    guess_letter = letter
                    return guess_letter

            # absolute fallback if frequency list exhausted or all letters guessed, picks the first available
            if unguessed:
                guess_letter = list(unguessed)[0] 
            else:
                guess_letter = 'a'
                
        return guess_letter

    def _get_lstm_probs(self, pattern, guessed_letters_set, model_to_use):
        """ Gets letter scores from the specified model's output probabilities. """
        model_to_use.eval()
        input_seq = torch.tensor([[CHAR_TO_IX.get(c, CHAR_TO_IX['_']) for c in pattern]], dtype=torch.long).to(self.device)
        visible_letters = {c for c in pattern if c.isalpha()}
        incorrect_guesses = guessed_letters_set - visible_letters
        state_vector_52d = torch.zeros(STATE_DIM_HISTORY, device=self.device)
        for char in visible_letters:
            if char in CHAR_TO_IX:
                char_idx = CHAR_TO_IX[char] - 2
                if 0 <= char_idx < ALPHABET_LEN: state_vector_52d[char_idx] = 1.0
        for char in incorrect_guesses:
            if char in CHAR_TO_IX:
                char_idx = CHAR_TO_IX[char] - 2
                if 0 <= char_idx < ALPHABET_LEN: state_vector_52d[char_idx + ALPHABET_LEN] = 1.0
        state_tensor = state_vector_52d.unsqueeze(0)

        with torch.no_grad():
            logits = model_to_use(input_seq, state_tensor)
            if logits.nelement() == 0 or logits.size(1) == 0:
                 print(f"Warning: Logits have zero elements or seq_len 0: {logits.shape}. Pattern: {pattern}")
                 return {}

            probabilities = torch.softmax(logits, dim=2).squeeze(0)           # shape (seq_len, vocab_size)

        # Stores {letter: max_prob across blanks}
        letter_probs = {} 
        if probabilities.nelement() > 0 and probabilities.dim() == 2:
            seq_len_prob = probabilities.size(0)
            pattern_len_actual = len(pattern)
            len_to_iterate = min(seq_len_prob, pattern_len_actual)

            for i in range(len_to_iterate):
                char_in_pattern = pattern[i]
                if char_in_pattern == '_':
                    # Iterate through valid alphabet indices (2 to 27 for a-z)
                    for letter_idx in range(2, VOCAB_SIZE):
                        letter = IX_TO_CHAR.get(letter_idx)
                        if letter:
                            # index is within bounds for probabilities tensor check
                            if letter_idx < probabilities.size(1):
                                prob = probabilities[i, letter_idx].item()
                                letter_probs[letter] = max(letter_probs.get(letter, 0.0), prob)

        return letter_probs

    def _filter_words_py(self, pattern: str, incorrect_guesses: set) -> list:
        """ Filters the full dictionary based on current game state. """
        valid_words = []
        pattern_len = len(pattern)
        # pre-compile the regex for efficiency, though I did not find that much perf gains
        try:
            # sanity checks
            safe_pattern = re.sub(r'[^a-z._]', '', pattern.replace('_', '.'))
            compiled_regex = re.compile(f"^{safe_pattern}$")
        except re.error as e:
            print(f"Regex error for pattern '{pattern}': {e}")
            return []

        # Pre-filter dictionary by length could be use to boost performance

        for word in self.full_dictionary: 
            # Check length first
            if len(word) == pattern_len:
                 # Check regex match
                 if compiled_regex.match(word):
                      # Check incorrect guesses
                      has_incorrect = False
                      for incorrect_char in incorrect_guesses:
                           if incorrect_char in word:
                                has_incorrect = True
                                break
                      if not has_incorrect:
                           valid_words.append(word)
        return valid_words

    def _get_dawg_probs(self, valid_words: list, guessed_letters: set) -> dict:
        """ Calculates softmax probability distribution over unguessed letters based on valid words. """
        dawg_probs = {}
        unguessed_chars = list(ALL_LETTERS - guessed_letters)
        num_unguessed = len(unguessed_chars)

        if not valid_words or num_unguessed == 0:
             # uniform probability if no valid words or all letters guessed
             return {char: 1.0/num_unguessed if num_unguessed > 0 else 0.0 for char in unguessed_chars}

        # frequency counting
        freq_counter = collections.Counter("".join(valid_words))

        # scores tensor only for unguessed letters
        scores_tensor = torch.zeros(num_unguessed, device=self.device) # Create on correct device
        char_to_tensor_idx = {char: i for i, char in enumerate(unguessed_chars)}

        for i, char in enumerate(unguessed_chars):
            scores_tensor[i] = freq_counter.get(char, 0)

        # softmax if there are scores > 0
        if torch.sum(scores_tensor) > 0:
            probs_tensor = torch.softmax(scores_tensor, dim=0)
            for i, char in enumerate(unguessed_chars):
                dawg_probs[char] = probs_tensor[i].item()
        else:
            # if all frequencies are 0, return uniform
            dawg_probs = {char: 1.0/num_unguessed for char in unguessed_chars}

        return dawg_probs

    def build_dictionary(self, dictionary_file_location):
        """ Loads and builds dict file. """
        try:
            with open(dictionary_file_location, "r") as text_file:
                full_dictionary = [line.strip().lower() for line in text_file if line.strip()]
            if not full_dictionary:
                 print(f"Warning: Dictionary loaded from {dictionary_file_location} is empty.")
            return full_dictionary
        except Exception as e:
             print(f"ERROR loading dictionary from {dictionary_file_location}: {e}")
             raise

    def start_game(self, practice=True, verbose=True):
        """ Starts and plays a single Hangman game. """
        # Reset for new game
        self.guessed_letters = [] 
        try:
            response = self.request("/new_game", {"practice": practice})
        except Exception as e:
             print(f"ERROR starting new game: {e}")
             return False

        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print(f"\nSuccessfully start game! ID: {game_id}. Tries: {tries_remains}. Word: {word}.")

            while tries_remains > 0:
                try:
                    guess_letter = self.guess(word)
                except Exception as e:
                     print(f"ERROR during guess generation for word '{word}': {e}")
                     # fallback
                     unguessed = list(ALL_LETTERS - set(self.guessed_letters))
                     if not unguessed: break
                     guess_letter = random.choice(unguessed)
                     print(f"Using random fallback guess: {guess_letter}")


                # validation and fallback for re-guess
                if not isinstance(guess_letter, str) or len(guess_letter) != 1 or not 'a' <= guess_letter <= 'z':
                    print(f"INTERNAL ERROR: Invalid guess '{guess_letter}'. Defaulting to 'a'.")
                    guess_letter = 'a'
                    if guess_letter in self.guessed_letters:                          # find first available
                         for char_code in range(ord('b'), ord('z') + 1):
                              char = chr(char_code)
                              if char not in self.guessed_letters:
                                   guess_letter = char
                                   break

                if guess_letter in self.guessed_letters:
                    if verbose: print(f"Warning: Attempting re-guess '{guess_letter}'. Using fallback.")
                    unguessed = list(ALL_LETTERS - set(self.guessed_letters))
                    if not unguessed:
                        print("Error: No valid letters left!")
                        break
                    # fallback - frequency list
                    found_fallback = False
                    for letter in "esiarntolcdugpmhbyfvwkxqz":
                         if letter in unguessed:
                             guess_letter = letter
                             found_fallback = True
                             break
                    if not found_fallback: guess_letter = unguessed[0] # Absolute fallback
                    if verbose: print(f"Fallback guess: {guess_letter}")

                # proceeds with the guess
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print(f"Guessing letter: {guess_letter}")

                try:
                    res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                    if verbose:
                        print(f"Server response: {res}")

                    status = res.get('status')
                    tries_remains = res.get('tries_remains', 0)               # default to 0 if missing
                    word = res.get('word', word)      # word update
                    message = res.get('message', '')

                    if status == "success":
                        if verbose: print(f"Successfully finished game: {game_id}")
                        return True
                    elif status == "failed":
                        reason = res.get('reason', 'Unknown reason')
                        if verbose: print(f"Failed game: {game_id}. Because: {reason} | Msg: {message}")
                        # Handle specific non-fatal failures from server
                        if 'already guessed' in message.lower():
                             print("Server reported already guessed. Correcting local state if needed.")
                             # Server state is truth, continue loop if tries remain
                             if guess_letter in self.guessed_letters: self.guessed_letters.remove(guess_letter)
                             # Don't return False here, try again
                        elif 'game is already over' in message.lower():
                            print("Server reported game over. Determining outcome from final state.")
                            return '_' not in word.replace(" ", "") # Win if no blanks
                        elif tries_remains <= 0 : # Ensure it's a loss due to tries
                            return False
                        else: # Other failure reason
                             print(f"Treating unexpected failure as loss: {reason}")
                             return False
                    elif status == "ongoing":
                        continue # Continue loop
                    else:
                        print(f"Warning: Unexpected server status '{status}'. Treating as failure.")
                        return False

                except HangmanAPIError as e:
                    print(f'HangmanAPIError on guess request: {e.message}. Game likely lost.')
                    return False
                except Exception as e:
                    print(f'Other exception on guess request: {e}.')
                    raise e

            # check final state if not already returned
            if tries_remains <= 0:
                 is_won = '_' not in word.replace(" ", "")
                 if verbose:
                      if is_won: print(f"Successfully finished game (on last try): {game_id}")
                      else: print(f"Failed game: {game_id}. Because: # of tries exceeded!")
                 return is_won
            else:
                 print("Warning: Loop ended unexpectedly.")
                 return False

        else:
            if verbose:
                print(f"Failed to start a new game. Server response: {response}")
            return False

    def my_status(self):
        """ Fetches game statistics from the API. """
        try:
            return self.request("/my_status", {})
        except Exception as e:
            print(f"Error fetching status: {e}")
            return [0, 0, 0, 0]

    # I added this robustness for 1000 runs
    def request(self, path, args=None, post_args=None, method=None):
        """ Makes a request to the Hangman API endpoint. """
        if args is None: args = dict()
        if post_args is not None: method = "POST"
        if self.access_token:
            if post_args and "access_token" not in post_args: post_args["access_token"] = self.access_token
            elif "access_token" not in args: args["access_token"] = self.access_token

        num_retry, time_sleep = 5, 2
        response = None
        last_exception = None

        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET", self.hangman_url + path,
                    timeout=self.timeout, params=args, data=post_args, verify=False
                )
                response.raise_for_status()

                # response parsing
                headers = response.headers
                content_type = headers.get('content-type', '').lower()

                if 'json' in content_type:
                    result = response.json()
                elif "access_token" in parse_qs(response.text): # Specific check for token response
                    query_str = parse_qs(response.text)
                    if "access_token" in query_str:
                        result = {"access_token": query_str["access_token"][0]}
                        if "expires" in query_str: result["expires"] = query_str["expires"][0]
                    else: 
                        # parses it as JSON if outer condition not met
                         try: result = response.json()
                         except json.JSONDecodeError: raise HangmanAPIError(f"Could not parse response: {response.text}")
                else:
                     try: result = response.json()
                     except json.JSONDecodeError:
                          raise HangmanAPIError(f'Unexpected Content-Type: {content_type}. Response: {response.text[:200]}') # Show start of text

                # error checks
                if isinstance(result, dict) and result.get("error"):
                    if "deactivated" in result.get("error_description", "").lower():
                         print("Account deactivated (expected after successful submission run).")
                         raise HangmanAPIError(result)
                    raise HangmanAPIError(result)

                # Allow dict or list as valid results
                if not isinstance(result, (dict, list)):
                    raise HangmanAPIError(f"Unexpected result format: {type(result)} - {str(result)[:200]}")

                return result

            except Exception as e:
                last_exception = e
                print(f"Unexpected Error during request/parsing (Attempt {it+1}/{num_retry}): {e}")

            # sleeps before next retry
            if it < num_retry - 1:
                time.sleep(time_sleep)
            else:
                print("Max retries exceeded.")
                if last_exception:
                     if isinstance(last_exception, requests.exceptions.HTTPError) and \
                        400 <= last_exception.response.status_code < 500 and \
                        not isinstance(last_exception, HangmanAPIError):
                          try: error_details = last_exception.response.json()
                          except json.JSONDecodeError: error_details = last_exception.response.text[:200]
                          raise HangmanAPIError(error_details if isinstance(error_details, dict) else str(error_details)) from last_exception
                     raise last_exception
                else:
                    raise HangmanAPIError("Max retries exceeded with no specific error captured.")

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        # parse error details
        if isinstance(result, dict):
            self.type = result.get("error_code", "")
            self.message = result.get("error_description")
            if not self.message:
                error_details = result.get("error")
                if isinstance(error_details, dict):
                    self.message = error_details.get("message")
                    self.code = error_details.get("code")
                    if not self.type:
                        self.type = error_details.get("type", "")
            if not self.message:
                self.message = result.get("error_msg")
            if not self.message:     
                 # fallbacks to string representation of dict
                 self.message = str(result)
        else:
             self.type = ""
             self.message = str(result)

        super().__init__(self.message)                 # calls the parent exception constructor

In [14]:
api = HangmanAPI(access_token="66f44cc9488917a85ff8affc6b59bc", timeout=2000)
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Loaded dictionary with 227300 words.
Initialized HYBRID strategy. Base Alpha = 0.365
Setting up device and seeding
Using device: cpu
Initializing BASE Hangman Model.
 -> Loading weights from /kaggle/input/hman-model-62-75/bilstm_attn_hangman_GATED_MLPGATE_finetuned_62.75.pth.
BASE model loaded successfully.
Initializing FINE-TUNED SHORT Hangman Model...
 -> Loading weights from /kaggle/input/hman-shortlen-model/bilstm_attn_hangman_GATED_MLPGATE_short_words_len5.pth.
SHORT model loaded successfully.

Successfully start game! ID: 77662074aa17. Tries: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '77662074aa17', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ e _ _ _ e _ '}
Guessing letter: r
Server response: {'game_id': '77662074aa17', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ e _ _ _ e _ '}
Guessing letter: s
Server response: {'game_id': '77662074aa17', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e _ e _ _ _ e _ '}
Guessing le

In [15]:
for i in range(1000):
    print('Playing ', i, ' th game')
    api.start_game(practice=0,verbose=False)
    
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

In [16]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status()
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.632
